In [12]:
import sys
from pathlib import Path

from utilities import get_default_data
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
import yaml

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount
from utils.contract_retrievers import retrieve_farm_by_address
from utils.utils_chain import get_token_details_for_address, get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken
from contracts.guild_contract import GuildContract
context = Context()

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()

2024-07-25 17:07:29,738 - WARNING - Structure definition for egld_wraps not found in /home/multiversx/Documents/guilds_py/deploy/configs-mainnet/deploy_structure.json
2024-07-25 17:07:29,741 - WARNING - Structure definition for pairs_view not found in /home/multiversx/Documents/guilds_py/deploy/configs-mainnet/deploy_structure.json
2024-07-25 17:07:29,743 - WARNING - Structure definition for stakings_boosted not found in /home/multiversx/Documents/guilds_py/deploy/configs-mainnet/deploy_structure.json
2024-07-25 17:07:29,745 - WARNING - Structure definition for metastakings_boosted not found in /home/multiversx/Documents/guilds_py/deploy/configs-mainnet/deploy_structure.json
2024-07-25 17:07:29,746 - WARNING - Structure definition for position_creator not found in /home/multiversx/Documents/guilds_py/deploy/configs-mainnet/deploy_structure.json


loaded 1 accounts from 1 PEM files.


2024-07-25 17:07:30,156 - DEBUG - Account.sync_nonce() done: 3663
2024-07-25 17:07:30,370 - DEBUG - Account.sync_nonce() done: 3663
2024-07-25 17:07:30,373 - INFO - Loaded 10 tokens.
2024-07-25 17:07:30,561 - DEBUG - Account.sync_nonce() done: 3663
2024-07-25 17:07:30,563 - INFO - Loaded 0 egld_wraps.
2024-07-25 17:07:30,566 - INFO - Loaded 1 locked_assets.
2024-07-25 17:07:30,568 - INFO - Loaded 1 proxies.
2024-07-25 17:07:30,569 - INFO - Loaded 0 simple_locks.
2024-07-25 17:07:30,571 - INFO - Loaded 1 simple_locks_energy.
2024-07-25 17:07:30,572 - INFO - Loaded 1 fees_collectors.
2024-07-25 17:07:30,573 - INFO - Loaded 1 unstakers.
2024-07-25 17:07:30,574 - INFO - Loaded 1 proxies_v2.
2024-07-25 17:07:30,575 - INFO - Loaded 0 router.
2024-07-25 17:07:30,576 - INFO - Loaded 1 router_v2.
2024-07-25 17:07:30,577 - INFO - Loaded 0 pairs.
2024-07-25 17:07:30,578 - INFO - Loaded 9 pairs_v2.
2024-07-25 17:07:30,579 - INFO - Loaded 0 pairs_view.
2024-07-25 17:07:30,580 - INFO - Loaded 0 farm

In [13]:
user = Account(pem_file="~/Documents/sh1.pem")
user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)

2024-07-25 17:07:32,659 - DEBUG - Account.sync_nonce() done: 2195


In [4]:
from utils.contract_retrievers import retrieve_guild_by_address


with open("config_file.yaml", "r") as file:
    config_file = yaml.safe_load(file)

guild_contract: GuildContract = retrieve_guild_by_address(config_file["deployedGuildAddress"])
guild_address = Address(config_file["deployedGuildAddress"])

ADVANCE EPOCHS

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, 1, 20)

SEND UTK

In [ ]:
import time
from multiversx_sdk import AccountNonceHolder, TokenPayment, TransactionComputer, UserSigner
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ContractDeploymentBuilder, MultiESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration


token = "UTK-2f80e9"
holder = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
holder.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)
holder.sync_nonce(context.network_provider.proxy)
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))


hashes = []
configs = DefaultTransactionBuildersConfiguration(chain_id="1")


transfers = [
    TokenPayment.meta_esdt_from_amount(token, 0, 100000, 18)
             ]

builder = MultiESDTNFTTransferBuilder(
        configs,
        destination=user.address,
        sender=holder.address,
        gas_limit=30000000,
        payments=transfers
    )

tx = builder.build()
tx.nonce = holder.nonce
tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = context.network_provider.proxy.send_transaction(tx)
print(hash)
hashes.append(hash)
time.sleep(2)

ISSUE TOKENS

In [ ]:
import time
from contracts.builtin_contracts import ESDTContract
esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
nr_tokens = 1
token_ticker = "UTKFARM"
holder = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
holder.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
holder.sync_nonce(context.network_provider.proxy)

# issue a number of nr_tokens tokens
hashes = []
# for i in range(1, nr_tokens+1):
hash = esdt_contract.issue_fungible_token(holder, context.network_provider.proxy, 
                                     [f"{token_ticker}", f"{token_ticker}", 1000000000000000000000000000, 18, 
                                      "canMint", "true", "canBurn", "true", "canPause", "true", "canFreeze", "true", "canWipe", "true", "canChangeOwner", "true", "canUpgrade", "true", "canAddSpecialRoles", "true"])
print(f"Issued {token_ticker} in: {hash}")
hashes.append(hash)
time.sleep(1)

# check issue success and retrieve tokens
issued_tokens = []
for hash in hashes:
    if context.network_provider.check_complex_tx_status(hash, "issue token"):
        tx_data = context.network_provider.proxy.get_transaction(hash)
        issue_event = tx_data.logs.find_first_or_none_event("issue")
        issued_token = str(issue_event.topics[0])
        print(issue_event.topics[0].raw)
        print("Issued token:", issued_token)
        issued_tokens.append(issued_token)
    else:
        print(f"Transaction {hash} failed!")

print(f'Issued ESDT Tokens: {issued_tokens}')

GUILD QUERY

In [10]:
from multiversx_sdk import SmartContractTransactionsFactory
from utilities import get_default_data
from utils.contract_data_fetchers import GuildContractDataFetcher

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()

# context.network_provider.proxy.get_data('getFarmTokenId')
guild_contract_address = Address(configuration["guildScAddressModel"])
GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getRewardTokenId")


'55544b2d326638306539'

PAUSE

In [ ]:

from contracts.farm_contract import FarmContract
# from contracts.guild_contract import GuildContract
user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

guild_contract: FarmContract = retrieve_farm_by_address(config_file["guildScAddressModel"])
# guild_contract.address = Address(config_file["deployedGuildAddress"])

tx_hash = guild_contract.pause(user, context.network_provider.proxy)

RESUME

In [ ]:
import yaml
from contracts.guild_contract import GuildContract


with open("config_file.yaml", "r") as file:
    config_file = yaml.safe_load(file)


user.sync_nonce(context.network_provider.proxy)
context.deployer_account.sync_nonce(context.network_provider.proxy)

guild_contract: GuildContract = retrieve_farm_by_address(config_file["guildScAddressModel"])
# guild_contract.address = Address(config_file["deployedGuildAddress"])

tx_hash = guild_contract.resume(user, context.network_provider.proxy)

START PRODUCE REWARDS

In [ ]:
import yaml
from contracts.guild_contract import GuildContract


with open("config_file.yaml", "r") as file:
    config_file = yaml.safe_load(file)

context.deployer_account.sync_nonce(context.network_provider.proxy)

guild_contract: GuildContract = retrieve_farm_by_address(config_file["deployedGuildAddress"])
# guild_contract.address = Address(config_file["deployedGuildAddress"])

tx_hash = guild_contract.start_produce_rewards(user, context.network_provider.proxy)

REGISTER FARM TOKEN

In [ ]:
import yaml
import time
from utilities import tx_success, get_default_data
from utils.contract_data_fetchers import FarmContractDataFetcher
from multiversx_sdk import TransactionComputer

guild_address = Address(config_file["deployedGuildAddress"])
token = "UTK-2f80e9"

(
    config,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()


register_farm_token_tx = sc_factory.create_transaction_for_execute(
    sender=test_address,
    contract=guild_address,
    function="registerFarmToken",
    gas_limit=70000000,
    arguments=[],
    # "UTKFARM", "UTKFARM", 18,""
    native_transfer_amount=50000000000000000
 
)
register_farm_token_tx.nonce = nonce_holder.get_nonce_then_increment()
register_farm_token_tx.signature = signer.sign(
    TransactionComputer().compute_bytes_for_signing(register_farm_token_tx)
)

result = provider.send_transaction(register_farm_token_tx)
print("Register Farm Token transaction hash:", result)
time.sleep(3)
tx_status = tx_success(result, provider)
if not tx_status:
    print("Register Farm Token transaction failed")
else:
    print("Farm token registered successfully")


Set admin

In [ ]:
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration
from multiversx_sdk import AccountNonceHolder, TokenPayment, TransactionComputer, UserSigner


newAdmin = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
newAdmin = Address("erd1qrat8pxccn8mj7n47v38uw7q4nkej6zq6mny3ljkj29uzdf6s42srfja7q")
config = DefaultTransactionBuildersConfiguration(chain_id="1")
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))
user.sync_nonce(context.network_provider.proxy)

builder = ContractCallBuilder(
    config,
    contract=Address(guild_contract.address),
    function_name="addAdmin",
    caller=user.address,
    nonce=user.nonce,
    call_arguments=[newAdmin],
    gas_limit=30000000
)

tx = builder.build()
tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = context.network_provider.proxy.send_transaction(tx)


In [ ]:
#GUILD FACTORY QUERIES
from multiversx_sdk import SmartContractTransactionsFactory
from utilities import get_default_data
from utils.contract_data_fetchers import GuildFactoryDataFetcher

(
    configuration,
    test_address,
    guild_factory_address,
    provider,
    network_config,
    signer,
    nonce_holder,
    sc_factory,
) = get_default_data()

# context.network_provider.proxy.get_data('getFarmTokenId')
guild_contract_address = Address(configuration["guildFactoryScAddress"])
GuildFactoryDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getConfigAddress")


TRANSFER TOKENS

In [ ]:
import time
from multiversx_sdk import AccountNonceHolder, TokenPayment, TransactionComputer, UserSigner
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ContractDeploymentBuilder, MultiESDTNFTTransferBuilder, DefaultTransactionBuildersConfiguration


token = "UTK-2f80e9"
holder = Account(pem_file="~/Documents/sh1.pem", pem_index=0)
holder.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)
holder.sync_nonce(context.network_provider.proxy)
user_signer = UserSigner.from_pem_file(Path("~/Documents/sh1.pem"))


hashes = []
configs = DefaultTransactionBuildersConfiguration(chain_id="1")


transfers = [
    TokenPayment.meta_esdt_from_amount(token, 0, 100000, 18)
             ]

builder = MultiESDTNFTTransferBuilder(
        configs,
        destination=user.address,
        sender=holder.address,
        gas_limit=30000000,
        payments=transfers
    )

tx = builder.build()
tx.nonce = holder.nonce
tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))

hash = context.network_provider.proxy.send_transaction(tx)
print(hash)
hashes.append(hash)
time.sleep(2)

STAKE

In [14]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent
from utils.utils_tx import ESDTToken

guild_address = Address(config_file["deployedGuildAddress"])

# user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)

farming_tk_balance = 0
farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farm tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

event = EnterFarmEvent(guild_contract.farmingToken, 0, 1000000000000000000000,
                       guild_contract.farmToken, farm_tk_nonce, farm_tk_balance)

tx = guild_contract.stakeFarm(
    context.network_provider,
    user, 
    event,
    True
)

print("Transaction hash:", tx)

2024-07-25 17:07:57,935 - DEBUG - Account.sync_nonce() done: 2195
2024-07-25 17:07:59,233 - INFO - enter farm
2024-07-25 17:07:59,235 - DEBUG - Account: erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd
2024-07-25 17:07:59,236 - INFO - Calling stakeFarm endpoint...
2024-07-25 17:07:59,237 - DEBUG - enter farm
2024-07-25 17:07:59,418 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.


Found 3 farm tokens in account


2024-07-25 17:07:59,637 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/7bd3f576d954ef4230d2c63230d45bee8e26a644842b2c6efb16a643655a24d1


Transaction hash: 7bd3f576d954ef4230d2c63230d45bee8e26a644842b2c6efb16a643655a24d1


UNSTAKE FROM GUILD

In [ ]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent, ExitFarmEvent
from utils.contract_retrievers import retrieve_guild_by_address
from utils.utils_tx import ESDTToken

guild_address = Address(config_file["deployedGuildAddress"])

user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address, context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
print(tokens_in_account)
for token in tokens_in_account:
    if int(token['balance']) > 0:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        event = ExitFarmEvent(guild_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')
        tx_hash = guild_contract.exitFarm(context.network_provider.proxy, user, event)
        # break


# 

UNBOND FARM

In [5]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent, ExitFarmEvent
from utils.contract_retrievers import retrieve_guild_by_address
from utils.utils_tx import ESDTToken
from utils.contract_data_fetchers import GuildContractDataFetcher, GuildFactoryDataFetcher


unbond_token = hex_to_string(GuildContractDataFetcher(guild_address,context.network_provider.proxy.url).get_data("getUnbondTokenId"))

# user.address = Address("erd1kvy5mas5z5ul3tt0ejvtrqcnrqqq6nwmwlxu7xpkqysnp3xwc06sgja0vu")
user.sync_nonce(context.network_provider.proxy)


unbond_tk_balance, unbond_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(unbond_token, user.address, context.network_provider.proxy)
print(tokens_in_account)

print(f'Found {len(tokens_in_account)} positions of {unbond_token} in account')
for token in tokens_in_account:
    if int(token['balance']) > unbond_tk_balance:
        unbond_tk_balance = int(token['balance'])
        unbond_tk_nonce = token['nonce']
        event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')
        tx_hash = guild_contract.unbondFarm(context.network_provider.proxy, user, event)
        # break

print(unbond_tk_balance)

# event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')

# tx_hash = guild_contract.unbondFarm(context.network_provider.proxy, user, event)

2024-07-25 14:13:50,060 - DEBUG - Account.sync_nonce() done: 2189


[]
Found 0 positions of UTKUNBND-ebe962 in account
0


CLAIM REWARDS

In [8]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user.address = Address("erd1kvy5mas5z5ul3tt0ejvtrqcnrqqq6nwmwlxu7xpkqysnp3xwc06sgja0vu")

user.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

        tx_hash = guild_contract.claimRewards(context.network_provider, user, event)
        # break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")



2024-07-25 14:18:40,481 - DEBUG - Account.sync_nonce() done: 2193
2024-07-25 14:18:41,794 - INFO - claimRewards
2024-07-25 14:18:41,796 - DEBUG - Account: erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd
2024-07-25 14:18:41,799 - DEBUG - claimRewards


Found 3 positions of UTKFARM-ebe962 in account


2024-07-25 14:18:42,043 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2024-07-25 14:18:42,301 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/8d5eddc2f0d814de71456b0816b53b6d67daf46e4f3820eeba18f8998b998faf


COMPOUND REWARDS

In [9]:
from events.farm_events import CompoundRewardsFarmEvent
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)


farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        # break
        event = CompoundRewardsFarmEvent(farm_tk_balance, farm_tk_nonce)

tx_hash = guild_contract.compoundRewards(context.network_provider, user, event)

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")



2024-07-25 14:30:11,182 - DEBUG - Account.sync_nonce() done: 2194
2024-07-25 14:30:12,542 - INFO - compoundRewards
2024-07-25 14:30:12,543 - DEBUG - Account: erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd
2024-07-25 14:30:12,543 - DEBUG - compoundRewards


Found 3 positions of UTKFARM-ebe962 in account


2024-07-25 14:30:12,828 - DEBUG - Contract call arguments: []
The transaction builders are deprecated and will be removed in a later release. Use the transaction factories instead.
2024-07-25 14:30:13,089 - INFO - No explorer known for https://proxy-shadowfork-three.elrond.ro. transaction raw path: https://proxy-shadowfork-three.elrond.ro/transaction/e67d2e13791028908e9c1d2bead9d0bc9f241046b73de53a9371673a611b18d6


MERGE TOKENS

In [ ]:
from events.farm_events import CompoundRewardsFarmEvent
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user.address = Address("erd1j8m8clpzxp4wpuv2hesw36whre6nysga9ggqrmy8n0shm4rgyues8pe5nd")
user.sync_nonce(context.network_provider.proxy)


farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = CompoundRewardsFarmEvent(1000000000, farm_tk_nonce)

tx_hash = guild_contract.compoundRewards(context.network_provider, user, event)

CALCULATE REWARDS FOR USER

In [16]:
from utils.utils_chain import Account, WrapperAddress as Address, base64_to_hex, decode_merged_attributes, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.contract_data_fetchers import GuildContractDataFetcher, GuildFactoryDataFetcher



user.sync_nonce(context.network_provider.proxy)
guild_contract_address = Address(configuration["deployedGuildAddress"])



farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(guild_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    # print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')
print(f'Found {len(tokens_in_account)} positions of {guild_contract.farmToken} in account')
attr = ""


for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']


rewards_per_share = GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getUserRewardPerShare")
compounded_reward = GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getAccumulatedRewards")
current_farm_amount = GuildContractDataFetcher(guild_contract_address,context.network_provider.proxy.url).get_data("getFarmTokenSupply")



# attributes_hex = base64_to_hex(tokens[token]['attributes'])
# decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), FARM_TOKEN_ATTRIBUTES)


# tx_hash = guild_contract.calculateRewardsForUser(context.network_provider, user, farm_tk_balance, attr) #hex value

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")
print(rewards_per_share)
print(compounded_reward)
print(current_farm_amount)




2024-07-19 12:20:18,693 - DEBUG - Account.sync_nonce() done: 2189


Farm Tokens in account:
	{'attributes': 'AAAAByr/uUbKHBoAAAAAAAAACTY1ya3F3qAAAA==', 'balance': '3999999000000000', 'creator': 'erd1qqqqqqqqqqqqqpgq7rkf7hsj7vk23ccugd7624020dkl6c65satsr78hnj', 'nonce': 11, 'royalties': '0', 'tokenIdentifier': 'UTKFARM-ebe962-0b', 'uris': ['']}
	{'attributes': 'AAAABz5Qftuv4jcAAAAAAAAACTY1ya3F3qAAAA==', 'balance': '1000000000000000000000', 'creator': 'erd1qqqqqqqqqqqqqpgq7rkf7hsj7vk23ccugd7624020dkl6c65satsr78hnj', 'nonce': 19, 'royalties': '0', 'tokenIdentifier': 'UTKFARM-ebe962-13', 'uris': ['']}
	{'attributes': 'AAAABz5egwBrDWQAAAAAAAAACIrHIwSJ6AAA', 'balance': '10000000000000000000', 'creator': 'erd1qqqqqqqqqqqqqpgq7rkf7hsj7vk23ccugd7624020dkl6c65satsr78hnj', 'nonce': 20, 'royalties': '0', 'tokenIdentifier': 'UTKFARM-ebe962-14', 'uris': ['']}
Found 3 positions of UTKFARM-ebe962 in account
17555365296803172
1388182191012176691215
502210003999999001342426


UPGRADE

In [ ]:
from multiversx_sdk import CodeMetadata
from utils.utils_tx import upgrade_call


guild_address = Address.from_bech32(configuration["guildConfigScAddressModel"])

context.deployer_account.address = user.address
context.deployer_account.sync_nonce(context.network_provider.proxy)
# hash = contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, config.DUMMY_PROXY, [])

metadata = CodeMetadata(upgradeable=True, payable_by_contract=True, readable=True)
gas_limit = 300000000

# if no_init:
#         arguments = []
# else:
arguments = []

bytecode = Path(
    "/home/multiversx/Documents/guilds_py/wasm/guild-sc-config.wasm"
    )

tx_hash = upgrade_call("guild", context.network_provider.proxy, gas_limit, context.deployer_account, guild_address,
                               bytecode, metadata, arguments)


END REWARDS

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

tx_hash = guild_contract.end_produce_rewards(user, context.network_provider.proxy)

In [ ]:
#GET GUILD CONFIG STATS

#GET GUILD FACTORY STATS

#GET GUILD SC STATS

guild_stats = guild_contract.get_all_stats(context.network_provider.proxy)

print(guild_stats)


farm_token_nonce, farm_token_amount = get_token(proxy_contract.proxy_farm_token, int(looking_for_nonce, 16))

farm_dec_attributes, lp_dec_attributes, lktk_dec_attributes = guild_contract.get_all_decoded_farm_token_attributes_from_api(context.network_provider.proxy, get_nonce(guild_contract.address))


CANCEL UNBOND FARM

In [ ]:
from contracts.guild_contract import GuildContract
from events.farm_events import EnterFarmEvent, ExitFarmEvent
from utils.contract_retrievers import retrieve_guild_by_address
from utils.utils_tx import ESDTToken
from utils.contract_data_fetchers import GuildContractDataFetcher, GuildFactoryDataFetcher


unbond_token = hex_to_string(GuildContractDataFetcher(guild_address,context.network_provider.proxy.url).get_data("getUnbondTokenId"))

# user.address = Address("erd1kvy5mas5z5ul3tt0ejvtrqcnrqqq6nwmwlxu7xpkqysnp3xwc06sgja0vu")
user.sync_nonce(context.network_provider.proxy)


unbond_tk_balance, unbond_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(unbond_token, user.address, context.network_provider.proxy)
print(tokens_in_account)

print(f'Found {len(tokens_in_account)} positions of {unbond_token} in account')
for token in tokens_in_account:
    if int(token['balance']) > unbond_tk_balance:
        unbond_tk_balance = int(token['balance'])
        unbond_tk_nonce = token['nonce']
        event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')
        tx_hash = guild_contract.cancelUnbondFarm(context.network_provider.proxy, user, event)
        # break

print(unbond_tk_balance)

# event = ExitFarmEvent(unbond_token, unbond_tk_balance, unbond_tk_nonce, '')

# tx_hash = guild_contract.unbondFarm(context.network_provider.proxy, user, event)

SET MIN/MAX STAKE

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

user.address = Address("erd1awuuva7dhnlun9huwx7uefwpkhzjx9d3phenujgkv3zvx9tvsatsw420qt")
user.sync_nonce(context.network_provider.proxy)

tx_hash = guild_contract.set(user, context.network_provider.proxy)

CLOSE GUILD